https://www.kaggle.com/code/ahmetgokkaya/pricing-derivatives-with-binomial-tree-model

In [1]:
import math
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import datetime as dt

# One step Binomial Model

### This doesnt do anything useful...

In [2]:
def binomial_model(S, u, d, p):
    """
    Returns a one-period binomial model.

    Parameters:
    S (float): initial asset price
    u (float): factor by which the asset price goes up
    d (float): factor by which the asset price goes down
    p (float): probability of an up movement

    Returns:
    (tuple): a tuple containing the following elements:
        - S_0 (float): the initial asset price
        - S_u (float): the asset price after an up movement
        - S_d (float): the asset price after a down movement
        - p (float): the probability of an up movement
    """
    S_0 = S
    S_u = S * u
    S_d = S * d
    return S_0, S_u, S_d, p

In [3]:
S_0, S_u, S_d, p = binomial_model(100, 1.2, 0.9, 0.5)
print(f"S_0 = {S_0}, S_u = {S_u}, S_d = {S_d}, p = {p}")

S_0 = 100, S_u = 120.0, S_d = 90.0, p = 0.5


# Two step Binomial Model

### Keep T and n the same for simplicity sake

In [123]:
# Define the parameters of the binomial tree
S0 = 100   # Initial stock price
K = 100    # Strike price
u = 1.1    # Up factor
d = 0.9    # Down factor
r = 0.0    # Risk-free rate
T = 1      # Time to maturity in years
n = 1      # Number of time steps

In [124]:
# Compute the values of the underlying asset at each node of the tree
stock_values = np.zeros((n+1, n+1))
for i in range(n+1):       # Loops through the columns
    for j in range(i+1):   # Loops through the rows
        stock_values[j,i] = S0 * (u**j) * (d**(i-j))

In [125]:
stock_values

array([[100.,  90.],
       [  0., 110.]])

In [126]:
# Compute the risk-neutral probabilities of moving up and down
p = (np.exp(r*T/n) - d) / (u - d)
q = 1 - p

In [127]:
print(p)
print(q)

0.4999999999999997
0.5000000000000002


In [128]:
# Compute the option values at each node of the tree
option_values = np.zeros((n+1, n+1))
for i in range(n+1):
    option_values[i,n] = max(stock_values[i,n] - K, 0)
for i in range(n-1, -1, -1):
    for j in range(i+1):
        option_values[j,i] = np.exp(-r*T/n) * (p*option_values[j,i+1] + q*option_values[j+1,i+1])


In [129]:
option_values

array([[ 5.,  0.],
       [ 0., 10.]])

In [70]:
# Print the option value at the initial node of the tree
print("Option value:", option_values[0,0])

Option value: 13.333333333333336


In [71]:
np.exp(0)

1.0

# Put example

In [51]:
def binomial_tree_put(S, K, r, sigma, T, N):
    """
    Calculates the price of a European put option using a two-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        sigma (float): stock price volatility
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):
        for j in range(i+1):
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])
            option_tree[j, i] = max(option_tree[j, i], K - stock_tree[j, i])

    return option_tree[0, 0]

In [58]:
# Example usage
S = 100  # initial stock price
K = 105  # strike price
r = 0.0  # risk-free interest rate
sigma = 0.2  # stock price volatility
T = 1  # time to maturity (in years)
N = 1  # number of time steps in the binomial tree

In [59]:
put_price = binomial_tree_put(S, K, r, sigma, T, N)
print("Put price:", put_price)

Put price: 12.71596944905797


# American Options

In [60]:
def binomial_tree_american_put(S, K, r, sigma, T, N):
    """
    Calculates the price of an American put option using a two-step binomial tree.

    Parameters:
        S (float): initial stock price
        K (float): strike price
        r (float): risk-free interest rate
        sigma (float): stock price volatility
        T (float): time to maturity (in years)
        N (int): number of time steps in the binomial tree

    Returns:
        (float) price of the put option
    """
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)

    # Generate the stock price tree
    stock_tree = np.zeros((N+1, N+1))
    for i in range(N+1):
        for j in range(i+1):
            stock_tree[j, i] = S * (u ** (i-j)) * (d ** j)

    # Generate the option value tree
    option_tree = np.zeros((N+1, N+1))
    option_tree[:, N] = np.maximum(K - stock_tree[:, N], 0)
    for i in range(N-1, -1, -1):
        for j in range(i+1):
            exercise_value = K - stock_tree[j, i]
            hold_value = np.exp(-r * dt) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])
            option_tree[j, i] = max(exercise_value, hold_value)

    return option_tree[0, 0]

In [61]:
# Example usage
S = 100  # initial stock price
K = 105  # strike price
r = 0.05  # risk-free interest rate
sigma = 0.2  # stock price volatility
T = 1  # time to maturity (in years)
N = 2  # number of time steps in the binomial tree

In [62]:
put_price = binomial_tree_american_put(S, K, r, sigma, T, N)
print("Put price:", put_price)

Put price: 9.088257638650122


# Binomial Tree Formulas

In [117]:
# Define the input values
S0 = 100      # Initial stock price
K = 100       # Strike Price
u = 1.1       # Up factor
d = 0.9       # Down factor
p = 0.50      # Probability of up move
r = 0.0      # Risk-free interest rate
T = 1         # Time to maturity (in years)
n = 1
dt = T / n

In [118]:
# Initialize the stock price tree
stock_tree = np.zeros((n+1, n+1))
stock_tree[0, 0] = S0

# Fill in the stock price tree
for i in range(1, n+1):
    for j in range(i+1):
        stock_tree[j, i] = S0 * u**(i-j) * d**j

In [119]:
# Print the stock price tree
print("Stock price tree:")
print(stock_tree)

Stock price tree:
[[100. 110.]
 [  0.  90.]]


In [120]:
# Initialize the option price tree
option_tree = np.zeros((n+1, n+1))

# Fill in the option price tree at the final nodes
for j in range(n+1):
    option_tree[j, n] = max(stock_tree[j, n] - K, 0)

# Work backwards through the tree to calculate the option prices at each node
for i in range(n-1, -1, -1):
    for j in range(i+1):
        option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])

In [121]:
# Print the option price tree
print("Option price tree:")
print(option_tree)

Option price tree:
[[ 5. 10.]
 [ 0.  0.]]


# Risk Neutral Valuation

In [162]:
# Define the input values
S0 = 100      # Initial stock price
K = 100       # Strike Price
u = 1.1       # Up factor
d = 0.9       # Down factor
r = 0.0       # Risk-free interest rate
T = 5         # Time to maturity (in years)
n = 5
dt = T / n

In [163]:
# Compute the risk-neutral probabilities of moving up and down
p = (np.exp(r*T/n) - d) / (u - d)
q = 1 - p

In [164]:
# Define the number of time steps and initialize the stock price tree
stock_tree = np.zeros((n+1, n+1))
stock_tree[0, 0] = S0

# Fill in the stock price tree
for i in range(1, n+1):
    for j in range(i+1):
        stock_tree[j, i] = S0 * u**(i-j) * d**j


# Print the stock price tree
print("Stock price tree:")
print(stock_tree)



Stock price tree:
[[100.    110.    121.    133.1   146.41  161.051]
 [  0.     90.     99.    108.9   119.79  131.769]
 [  0.      0.     81.     89.1    98.01  107.811]
 [  0.      0.      0.     72.9    80.19   88.209]
 [  0.      0.      0.      0.     65.61   72.171]
 [  0.      0.      0.      0.      0.     59.049]]


In [165]:
# Initialize the option price tree
option_tree = np.zeros((n+1, n+1))

# Fill in the option price tree at the final nodes
for j in range(n+1):
    option_tree[j, n] = max(stock_tree[j, n] - K, 0)

# Work backwards through the tree to calculate the option prices at each node
for i in range(n-1, -1, -1):
    for j in range(i+1):
        option_tree[j, i] = np.exp(-r * dt) * (p * option_tree[j, i+1] + q * option_tree[j+1, i+1])

# Print the option price tree
print("Option price tree:")
print(option_tree)



Option price tree:
[[ 9.3126875 14.6870625 22.473875  33.1       46.41      61.051    ]
 [ 0.         3.9383125  6.90025   11.84775   19.79      31.769    ]
 [ 0.         0.         0.976375   1.95275    3.9055     7.811    ]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         0.       ]]


In [166]:
# Print the option price at time 0
print("Option price:", option_tree[0, 0])

Option price: 9.31268750000001


In [167]:
np.exp(0)

1.0